## Este notebook es encargado utilizar el CSV con las prruebas consolidadas para crear y entrenar un modelo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn import preprocessing
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
biometrics_df = pd.read_csv("./1_standarized_biometrics.csv")
biometrics_df.head()

,HR,HRV,MicroSiemens,ArousalMean
0,-0.255155,-1.609745,-1.658083,0.0
1,0.427759,-1.627232,-1.646859,0.0
2,-0.088221,-1.627232,-1.658083,0.0
3,-0.133748,-1.627232,-1.658083,0.0
4,-0.224803,-1.443622,-1.669307,0.0


In [3]:
X = biometrics_df[['MicroSiemens', 'HR', 'HRV' ]].values
y = biometrics_df['ArousalMean'].values

In [4]:
X

array([[-1.65808285, -0.25515501, -1.60974495],
       [-1.64685869,  0.42775886, -1.62723153],
       [-1.65808285, -0.08822051, -1.62723153],
       ...,
       [ 2.0914088 ,  0.28619676,  0.52922629],
       [ 2.0914088 ,  1.18911296,  0.52922629],
       [ 2.0914088 ,  0.66241185,  0.52922629]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

In [6]:
# Para poder usar un clasificador, los datos de la variable target deben ser discretos.
#Los convertimos a True-False (Aroused-NotAroused) con el fin de poder entrear un clasificador binario
def map_to_0_and_1(arousal):
    return 0 if arousal < 5 else 1

y_train_aroused = np.array(list(map(map_to_0_and_1, y_train)))
y_test_aroused = np.array(list(map(map_to_0_and_1, y_test)))

# Models Creation

In [7]:
# model = "adaboost"
# model = "knn"
# model = "random-forest"
# model = "svm-rbf"
model = "svm-poly"

### Adaboost

In [8]:
if model == "adaboost":
    clf = AdaBoostClassifier(n_estimators=100, random_state=42)
    #Adaboost with RandomForest
    clf2 = AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
    print(f"Usando {model}")

### KNN

In [9]:
if model == "knn":
    clf = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')

    # Grid Search
    clf.get_params()
    params = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
    grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, verbose=1)
    print(f"Usando {model}")

### Random Forest

In [10]:
if model == "random-forest":
    clf = RandomForestClassifier(random_state=42, n_estimators=100)

    # Grid Search
    rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
    params_grid = {"max_depth": [3, None], "min_samples_split": [2, 3, 10], "min_samples_leaf": [1, 3, 10], "bootstrap": [True, False], "criterion": ['gini', 'entropy']}
    grid_search_cv = GridSearchCV(rf_clf, params_grid, n_jobs=-1, cv=5, verbose=1, scoring='accuracy', iid=False)
    print(f"Usando {model}")

### SVM rbf

In [11]:
if model == "svm-rbf":
    clf = svm.SVC(C=0.1, gamma=0.01, kernel='rbf')
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='rbf', C=1, gamma=0.1))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")

### SVM Poly

In [12]:
if model == "svm-poly":
    clf = svm.SVC(kernel='poly', degree=3)
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='poly'))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1), 'clf__degree': (3, 4, 5)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")

Usando svm-poly


# Model Fit

In [ ]:
clf.fit(X_train, y_train_aroused)
grid_search_cv.fit(X_train, y_train_aroused)

Fitting 3 folds for each of 147 candidates, totalling 441 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min


# Performance Measurement

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))        
        
        res = cross_val_score(clf, X_test, y_test, cv=10, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

## Model performance

In [ ]:
print_score(clf, X_train, y_train_aroused, X_test, y_test_aroused, train=True)
print("\n******************************\n")
print_score(clf, X_train, y_train_aroused, X_test, y_test_aroused, train=False)

## Grid Search performance

In [ ]:
y_test_pred_grid = grid_search_cv.predict(X_test)

print_score(grid_search_cv, X_train, y_train_aroused, X_test, y_test_aroused, train=True)
print("\n******************************\n")
print_score(grid_search_cv, X_train, y_train_aroused, X_test, y_test_aroused, train=False)

grid_search_cv.best_estimator_
grid_search_cv.best_estimator_.get_params()
grid_search_cv.best_params_
grid_search_cv.cv_results_

# Save model to disk

In [ ]:
import pickle
# save the model to disk
pickle.dump(grid_search_cv, open('./models/model', 'wb'))
print('Done')